In [ ]:
Create an aglorithm that makes all x where j is played with
in while look
#TEST 4: Classical Vehicle

# ABSTRACT --------------------------------------------------------------------------------------------------------------------------------
- Even though we are using quantum mechanics principles to solve these problems, a vehicle cannot exist in two places at once. 
My prototype has several "exactly once" constraints, such as visiting a customer once or starting from the depot once. This
test is to verify that each “exactly-one” constraint (such as visiting a customer once or starting from the depot once) is correctly 
encoded in the QUBO model.

# METHOD 
- To make sure this works, we will try three different cases. First, we will choose a location to never visit, then visit once,
and then visit several times. The cost for the first and last cases should be unfeasible. The algorithm should run normally, 
but we will alter the variables of that location. To create the three different cases, below are all the variable combinations
that must have a sum of 1:

ConstraintVar = {
  "VisitCustomer_1": ["x_0_1_a","x_2_1_a","x_0_1_b","x_2_1_b"],
  "VisitCustomer_2": ["x_0_2_a","x_1_2_a","x_0_2_b","x_1_2_b"],
  "StartVehicle_a": ["x_0_1_a","x_0_2_a"],
  "StartVehicle_b": ["x_0_1_b","x_0_2_b"],
  "EndVehicle_a": ["x_1_0_a","x_2_0_a"],
  "EndVehicle_b": ["x_1_0_b","x_2_0_b"]
}

# ADDED CODE--------------------------------------------------------------------------------------------------------------------------------
ConstraintVar = {
  "VisitCustomer_1": ["x_0_1_a","x_2_1_a","x_0_1_b","x_2_1_b"],
  "VisitCustomer_2": ["x_0_2_a","x_1_2_a","x_0_2_b","x_1_2_b"],
  "StartVehicle_a": ["x_0_1_a","x_0_2_a"],
  "StartVehicle_b": ["x_0_1_b","x_0_2_b"],
  "EndVehicle_a": ["x_1_0_a","x_2_0_a"],
  "EndVehicle_b": ["x_1_0_b","x_2_0_b"]
}

print("incoming vars to customer 1:", ConstraintVar["VisitCustomer_1"])
print("count:", len(ConstraintVar["VisitCustomer_1"]))
samples = {
    "Valid": {v: 1 if i == 0 else 0 for i, v in enumerate(ConstraintVar["VisitCustomer_1"])},  
    "UnderSelected": {v: 0 for v in ConstraintVar},                               
    "OverSelected": {v: 1 if i < 2 else 0 for i, v in enumerate(ConstraintVar["VisitCustomer_1"])}   
}

Variables = list(bqm.variables)
for s in samples.values():
    for v in Variables:
        if v not in s:
            s[v] = 0

for name, sample in samples.items():
    E = bqm.energy(sample)
    print(f"{name}: {E:.2f}")

# RESULTS-----------------------------------------------------------------------------------------------------------------------------------
Visit customer once: ✅
Start from Depot: ✅
Finish at Depot: ❌
# CONCLUSION--------------------------------------------------------------------------------------------------------------------------------
The prototype successfully penalizes visiting a customer more than once, and not starting from the depot. However,
it fails to force the solver to finish at the depot. Hence this part needs to be checked and fixed moving forward.

- It is worth documenting that the prototype first failed this test due to logical errors in the added code. These were fixed.

In [62]:
import dimod
from dimod import ConstrainedQuadraticModel, Binary
from dimod import quicksum

# Depot, Customers 1 and 2, Vehicles a and b
nodes = [0, 1, 2]          # 0 = depot
customers = [1, 2]
vehicles = ['a', 'b']

# Time dictionary (cost)
t = {
    (0,1): 3, (1,0): 3,
    (0,2): 5, (2,0): 5,
    (1,2): 4, (2,1): 4
}


# Create binary decision variables                   
# ---------------------------------
x = {}
for i in nodes:
    for j in nodes:
        if i != j: 
            for k in vehicles:
                name = f"x_{i}_{j}_{k}"
                x[i,j,k] = Binary(name)


# Building the CQM
# -----------------
cqm = ConstrainedQuadraticModel()

# 1) Objective: minimize total travel time
objective = quicksum(t[i,j] * x[i,j,k]
                     for i,j,k in x.keys())
cqm.set_objective(objective)

# Constraints
# -------------

# Each customer visited once
for v in customers:
    cqm.add_constraint(
        quicksum(x[i,v,k] for i in nodes if i != v for k in vehicles) == 1,
        label=f"Visit_{v}"
    )

# Start and end at Depot for each vehicle
for k in vehicles:
    # start from depot once
    cqm.add_constraint(quicksum(x[0,j,k] for j in customers) == 1, label=f"Start_{k}", weight=50)
    # return to depot once
    cqm.add_constraint(
        quicksum(x[i,0,k] for i in customers) - quicksum(x[0,j,k] for j in customers) == 0,
        label=f"depot_flow_{k}"
    )

# Flow conservation
for v in customers:
    for k in vehicles:
        cqm.add_constraint(
            quicksum(x[i,v,k] for i in nodes if i != v) -
            quicksum(x[v,j,k] for j in nodes if j != v) == 0,
            label=f"flow_{v}_{k}"
        )


# Converting to BQM 
bqm, invert = dimod.cqm_to_bqm(cqm)




# ----TEST-------------------------------------------------------------------------------
ConstraintVar = {
  "VisitCustomer_1": ["x_0_1_a","x_2_1_a","x_0_1_b","x_2_1_b"],
  "VisitCustomer_2": ["x_0_2_a","x_1_2_a","x_0_2_b","x_1_2_b"],
  "StartVehicle_a": ["x_0_1_a","x_0_2_a"],
  "StartVehicle_b": ["x_0_1_b","x_0_2_b"],
  "EndVehicle_a": ["x_1_0_a","x_2_0_a"],
  "EndVehicle_b": ["x_1_0_b","x_2_0_b"]
}

samples = {
    "Valid": {v: 1 if i == 0 else 0 for i, v in enumerate(ConstraintVar["EndVehicle_b"])},  
    "UnderSelected": {v: 0 for v in ConstraintVar["EndVehicle_b"]},                               
    "OverSelected": {v: 1 for v in ConstraintVar["EndVehicle_b"]} 
}

Variables = list(bqm.variables)
for s in samples.values():
    for v in Variables:
        if v not in s:
            s[v] = 0

for name, sample in samples.items():
    E = bqm.energy(sample)
    print(f"{name}: {E:.2f}")
# ---------------------------------------------------------------------------------------


Valid: 303.00
UnderSelected: 200.00
OverSelected: 508.00
